# Program for training the image orientation model

### import stuff

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

### import tensorflow for the machine learning model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

In [ ]:
### function that takes an image and returns many permutations of it in different orientations and lightings

### initialize the data

In [ ]:
data_dir = "data"
augmented_data_dir = "augmented-data"
# get all images from the dir and create 4 versions, one in each orientation